In [13]:
from glob import glob
import pandas as pd
import gdown
import warnings
warnings.filterwarnings('ignore')

In [2]:
#downloading datasets
url='https://drive.google.com/drive/folders/1u_l-QFGwAxZ4nxmNJVScgD7zOnKVEhjp'

gdown.download_folder(url,quiet=True, remaining_ok=True, use_cookies=False)

['/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Benin.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Burkina_Faso.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Cape_Verde.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Gambia.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Ghana.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Guinea_Bissau.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Guinea.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Ivory_Coast.csv',
 '/Users/elohorokpako/Desktop/Study_articles/Programming/Hamoye_capstone_project/New Dataset/Liberia.csv',
 '/Users/elohorokpako/Des

In [2]:
files=glob('New Dataset/*')
files

['New Dataset/Burkina_Faso.csv',
 'New Dataset/Ivory_Coast.csv',
 'New Dataset/Niger.csv',
 'New Dataset/Togo.csv',
 'New Dataset/Gambia.csv',
 'New Dataset/Mali.csv',
 'New Dataset/Guinea_Bissau.csv',
 'New Dataset/Benin.csv',
 'New Dataset/Nigeria.csv',
 'New Dataset/Senegal.csv',
 'New Dataset/Sierra_Leone.csv',
 'New Dataset/Liberia.csv',
 'New Dataset/Ghana.csv',
 'New Dataset/Cape_Verde.csv',
 'New Dataset/Guinea.csv']

In [3]:
frames=[pd.read_csv(file) for file in files]

In [4]:
def creating_dataframe(df,x):
    '''
    Parameters: df (dataframe from the frames generated)
    x: the position of the file in the glob files generated
    
    Returns: a dataframe with the stations metadata and stations average temperature from 1980 - 2022
    '''
    #pivot the data to have the index as date, to resample to Daily reading
    data=pd.pivot_table(df, values='TAVG',index='DATE',columns='STATION')

    #remove the names of the columns
    data.columns.name=None

    #convert the index to datetime
    data.index=pd.to_datetime(data.index)

    #drop stations with missing values above 30%
    missing_values=data.isna().mean()[data.isna().mean()>.3].index
    data.drop(columns=missing_values,inplace=True)

    #fill the missing values with a ffill and bfill method
    data=data.ffill().bfill()

    #resample the data to daily reading, inorder to have a dataframe with same length when merging all of them
    data=data.resample('D').mean().fillna(method='ffill')
    
    #convert fahrenheit reading to celcius
    data=(data-32) *(5/9)
    data=data.astype(float).round(2)
    
    #transpose the data to have the index as Stations in order to merge the metadata of the stations
    transpose_data=data.T
    transpose_data.columns.name=None
    transpose_data.index.name='STATION'
    
    #get the stations
    stations=transpose_data.index
    
    #the name of the columns for the metadata
    names=['LATITUDE','LONGITUDE','ELEVATION']
    
    #getting the metadata from the dataframe
    metadata=df.groupby('STATION')[names].mean().loc[stations].reset_index()
    
    #merging the metadata and the daily readings into one dataframe
    df1=pd.merge(metadata,transpose_data,
        left_on='STATION',
        right_on=transpose_data.index,
        how='inner')
    
    #inserting the country name for that dataframe
    df1.insert(4,'COUNTRY',files[x])
    
    #cleaning the country name
    df1['COUNTRY']=(df1['COUNTRY'].str.replace('New Dataset/','').
                str.replace('.csv','',regex=True).
                str.replace('_',' ',regex=True))
    return df1

In [5]:
#generate new frames to store the dataframes after applying the function
new_frames=[creating_dataframe(frames[i],i) for i in range(len(files))]

In [6]:
#concatenate the frames
dataframe=pd.concat(new_frames,ignore_index=True)

dataframe.head()

,STATION,LATITUDE,LONGITUDE,ELEVATION,COUNTRY,1980-01-01 00:00:00,1980-01-02 00:00:00,1980-01-03 00:00:00,1980-01-04 00:00:00,1980-01-05 00:00:00,...,2022-12-22 00:00:00,2022-12-23 00:00:00,2022-12-24 00:00:00,2022-12-25 00:00:00,2022-12-26 00:00:00,2022-12-27 00:00:00,2022-12-28 00:00:00,2022-12-29 00:00:00,2022-12-30 00:00:00,2022-12-31 00:00:00
0,UV000005502,13.580,-2.430,340.0,Burkina Faso,23.89,24.44,24.44,23.33,23.89,...,24.44,23.33,23.33,23.33,22.78,25.00,25.56,25.00,25.00,24.44
1,UV000005507,12.070,0.350,301.0,Burkina Faso,24.44,24.44,25.00,25.00,25.56,...,23.33,23.33,22.78,22.78,22.78,24.44,25.00,24.44,24.44,23.33
2,UV000005522,10.330,-3.180,333.0,Burkina Faso,26.67,27.22,26.11,26.11,27.22,...,25.00,25.00,23.89,23.33,22.78,24.44,25.56,25.56,25.56,25.00
3,UV000065501,14.033,-0.033,277.0,Burkina Faso,21.67,24.44,22.78,23.33,23.89,...,32.22,32.22,32.22,32.22,32.22,32.22,32.22,32.22,32.22,32.22
4,UV000065516,11.750,-2.933,271.0,Burkina Faso,24.44,26.11,25.00,24.44,25.56,...,23.89,23.89,22.78,22.78,22.22,24.44,25.00,25.00,23.89,23.33


In [7]:
#fill the missing values (without entry data for the selected dates) using a backfill
dataframe=dataframe.fillna(method='bfill',axis=1)

In [8]:
dataframe.isna().mean()

STATION                0.0
LATITUDE               0.0
LONGITUDE              0.0
ELEVATION              0.0
COUNTRY                0.0
                      ... 
2022-12-27 00:00:00    0.0
2022-12-28 00:00:00    0.0
2022-12-29 00:00:00    0.0
2022-12-30 00:00:00    0.0
2022-12-31 00:00:00    0.0
Length: 15711, dtype: float64

In [10]:
dataframe.to_csv('West_Africa_temperature_1980_2022.csv',index=False)

#### Save the station metadata

In [14]:
#get the station metadata, which are the first 5 columns, [Station, Lat, Lon, Elevation, Country]
station_metadata=dataframe[['STATION','LATITUDE','LONGITUDE','ELEVATION','COUNTRY']]

#get the mean temperature for the years from 1980-2022 for all each station
mean_temp=dataframe.iloc[:,5:].mean(axis=1)

#assign this column to as a new column in the station metadata
station_metadata['AVG TEMP']=mean_temp

In [15]:
station_metadata

,STATION,LATITUDE,LONGITUDE,ELEVATION,COUNTRY,AVG TEMP
0,UV000005502,13.58,-2.43,340.0,Burkina Faso,29.210733
1,UV000005507,12.07,0.35,301.0,Burkina Faso,28.545106
2,UV000005522,10.33,-3.18,333.0,Burkina Faso,27.652866
3,UV000065501,14.033,-0.033,277.0,Burkina Faso,29.576589
4,UV000065516,11.75,-2.933,271.0,Burkina Faso,28.511006
...,...,...,...,...,...,...
67,SL000061856,8.617,-13.2,27.0,Sierra Leone,27.166296
68,LIM00065660,6.234,-10.362,9.4,Liberia,27.302027
69,GHM00065472,5.605,-0.167,62.5,Ghana,27.472014
70,CVM00008594,16.741,-22.949,53.9,Cape Verde,23.758664


In [17]:
#save this metadata
station_metadata.to_csv('Station_metadata.csv',index=False)